# Carregando os dados

In [2]:
import pandas as pd

In [3]:
%%time
transactions_2009_2010 = pd.read_excel('../online_retail_II.xlsx',sheet_name="Year 2009-2010", engine='openpyxl')
transactions_2010_2011= pd.read_excel('../online_retail_II.xlsx',sheet_name="Year 2010-2011", engine='openpyxl')

CPU times: total: 1min 58s
Wall time: 2min 1s


In [4]:
# Concatenar os DataFrames (empilhar as linhas)
transactions = pd.concat([transactions_2009_2010, transactions_2010_2011], ignore_index=True)

# Exibir as primeiras linhas do DataFrame combinado
print(transactions.shape)

(1067371, 8)


In [5]:
transactions.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


# Limpeza de Dados

In [23]:
# Verificando valores ausentes
transactions.isna().sum()

transactions['isCancelled'] = transactions['Invoice'][0]

# Verificando valores númericos
transactions['Invoice'] = pd.to_numeric(transactions['Invoice'], errors='coerce')

transactions['InvoiceDate'] = pd.to_datetime(transactions['InvoiceDate'])

# Análise de Dados Básicos

In [24]:
# Adiciona a coluna ano
transactions['ano'] = transactions['InvoiceDate'].dt.year

# Remove as transações canceladas
transactions_filtered = transactions[~transactions['Invoice'].astype(str).str.contains('C', na=False)]

# Agrupoar por produto e somar as quantidades vendidas
top_products = transactions_filtered.groupby('Description')['Quantity'].sum().reset_index()

# Ordenar do mais vendido para o menos vendido
top_products = top_products.sort_values(by='Quantity', ascending=False)

# Mostra os 10 produtos mais vendidos nos 4 anos
print(top_products.head(10))

                             Description  Quantity
5388   WORLD WAR 2 GLIDERS ASSTD DESIGNS    108545
5266  WHITE HANGING HEART T-LIGHT HOLDER     93050
328        ASSORTED COLOUR BIRD ORNAMENT     81306
2508             JUMBO BAG RED RETROSPOT     78090
801                  BROCADE RING PURSE      70700
3204  PACK OF 60 PINK PAISLEY CAKE CASES     56575
168          60 TEATIME FAIRY CAKE CASES     54366
4706                SMALL POPCORN HOLDER     49616
3207     PACK OF 72 RETROSPOT CAKE CASES     49344
3206    PACK OF 72 RETRO SPOT CAKE CASES     46106


In [28]:
# Mostrar o total de vendas
transactions_filtered['Invoice'].unique().shape[0]

# Total de vendas por ano
transactions_by_year = transactions_filtered.groupby('ano').head()
print(transactions_by_year)


         Invoice StockCode                          Description  Quantity  \
0       489434.0     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434.0    79323P                   PINK CHERRY LIGHTS        12   
2       489434.0    79323W                  WHITE CHERRY LIGHTS        12   
3       489434.0     22041         RECORD FRAME 7" SINGLE SIZE         48   
4       489434.0     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
45228   493410.0   TEST001              This is a test product.         5   
45229        NaN     21539              RETRO SPOTS BUTTER DISH        -1   
45230   493412.0   TEST001              This is a test product.         5   
45231   493413.0     21724      PANDA AND BUNNIES STICKER SHEET         1   
45232   493413.0     84578       ELEPHANT TOY WITH BLUE T-SHIRT         1   
567942  539993.0     22386              JUMBO BAG PINK POLKADOT        10   
567943  539993.0     21499                   BLUE POLKADOT WRAP        25   